Imports

In [1]:
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
import torch

import cv2

import pandas as pd
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict
from tqdm import tqdm
import copy
import random


Bad key text.latex.unicode in file /Users/Mikhail.Shagvaliev/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /Users/Mikhail.Shagvaliev/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /Users/Mikhail.Shagvaliev/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably ne

In [2]:
from bricks_dataset import BrickDataset

In [3]:
DATASET_TRAIN_CSV = '..//data//description//train.csv'
DATASET_VALIDATE_CSV = '..//data//description//test.csv'

Get the mean and std for train data

In [12]:
transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)
dataset_train = BrickDataset(DATASET_TRAIN_CSV, "", transform)
train_loader = DataLoader(dataset_train, batch_size=32)

In [13]:
def get_mean_w_std(loader):
    batch_count, mean, mean_squared = 0, 0, 0
    for image, _ in loader:
        image = image.float()
        batch_count += 1
        mean += torch.mean(image, dim=[0, 2, 3])
        mean_squared += torch.mean(image**2, dim=[0, 2, 3])

    mean /= batch_count
    mean_squared /= batch_count
    return mean, (mean_squared - mean**2) ** 0.5

In [14]:
mean, std = get_mean_w_std(train_loader)
print(
    {
        'mean': mean,
        'std': std
     }
)

{'mean': tensor([0.6113, 0.6691, 0.7893]), 'std': tensor([0.2782, 0.2572, 0.2881])}


In [16]:
def rotation(image):
  return TF.rotate(image, random.choice([0, 90, 180, 270]))

train_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        rotation
    ]
)

val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

In [17]:
train_dataset = BrickDataset(DATASET_TRAIN_CSV, "", train_transform)
val_dataset = BrickDataset(DATASET_VALIDATE_CSV, "", val_transform)

In [18]:
class MetricMonitor:
    def __init__(self, float_precision=4):
      self.float_precision = float_precision

      self.metrics = None
      self.reset()

    def reset(self):
      self.metrics = defaultdict(lambda: {'val': 0, 'count': 0, 'avg': 0})

    def update(self, metric_name, val):
      metric = self.metrics[metric_name]

      metric['val'] += val
      metric['count'] += 1
      metric['avg'] = metric['val'] / metric['count']

    def __str__(self):
      return ' | '.join(
          [
              f'{metric_name}: {format(metric["avg"], f".{self.float_precision}f")}'
              for (metric_name, metric) in self.metrics.items()
          ]
      )

In [19]:
def output_to_prediction(net_output):
    probabilities = nn.Softmax()(net_output)
    pred = torch.argmax(probabilities)

    return pred

In [20]:
class ModelTrainAndVal:
  def __init__(self, model, params, train_dataset, val_dataset, model_name):
    self.params = params

    self.model = model.to(self.params['device'])

    self.train_dataset = train_dataset
    self.val_dataset = val_dataset

    self.model_name = model_name
    self.train_metrics = []
    self.val_metrics = []

  def train_epoch(self, train_loader, epoch, metric_monitor, criterion, optimizer):
    metric_monitor.reset()
    self.model.train()
    stream = tqdm(train_loader)
    for i, (images, target) in enumerate(stream, start=1):
        images = images.float()
        target = torch.tensor([target]).long()

        images = images.to(self.params['device'], non_blocking=True)
        target = target.to(self.params['device'], non_blocking=True)

        output = self.model(images)

        loss = criterion(output, target)
        metric_monitor.update('Loss', loss.item())

        with torch.no_grad():
          pred = output_to_prediction(output)
          correct += (pred == target).float().sum()

          metric_monitor.update('Accuracy', correct)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        stream.set_description(
            f'Epoch: {epoch}. Train.    {metric_monitor}'
        )

  def validate_epoch(self, val_loader, epoch, metric_monitor, criterion):
    metric_monitor.reset()
    self.model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
      correct = 0.0
      for i, (images, target) in enumerate(stream, start=1):
        images = images.float()
        target = torch.tensor([target]).long()

        images = images.to(self.params['device'], non_blocking=True)
        target = target.to(self.params['device'], non_blocking=True)

        output = self.model(images)

        loss = criterion(output, target)
        metric_monitor.update('Loss', loss.item())

        with torch.no_grad():
          pred = output_to_prediction(output)
          correct += (pred == target).float().sum()

          metric_monitor.update('Accuracy', correct)

        stream.set_description(
            f'Epoch: {epoch}. Validate. {metric_monitor}'
        )

  def train(self, epoch_num):
    train_loader = DataLoader(
        self.train_dataset,
        batch_size=self.params['batch_size'],
        num_workers=2,
        pin_memory=True
    )
    val_loader = DataLoader(
        self.val_dataset,
        batch_size=self.params['batch_size'],
        pin_memory=True
    )

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.model.parameters(), lr=self.params['learning_rate'])

    saved_num = 0
    for epoch in range(1, epoch_num+1):
      train_metric_monitor = MetricMonitor()
      self.train_epoch(train_loader, epoch, train_metric_monitor, criterion, optimizer)
      self.train_metrics.append(train_metric_monitor.metrics)

      val_metric_monitor = MetricMonitor()
      self.validate_epoch(val_loader, epoch, val_metric_monitor, criterion)
      self.val_metrics.append(val_metric_monitor.metrics)

      # if val_metric_monitor.metrics['Accuracy']['avg'] >= self.SAVE_THRESHOLD and saved_num <= self.LIMIT_SAVED:
      #   saved_num += 1
      #   torch.save(
      #       self.model.state_dict(),
      #       f'{self.PATH_MODELS_SAVES}{self.model_name}-e{epoch}' +
      #       f'-v_acc{val_metric_monitor.metrics["Accuracy"]["avg"]}' +
      #       f'-t_acc{train_metric_monitor.metrics["Accuracy"]["avg"]}'
      #   )

    return self.model

In [21]:
model = models.squeezenet1_1(pretrained=True)

In [22]:
model.classifier[1] = torch.nn.Conv2d(in_channels=512, out_channels=len(BrickDataset.ALL_LABELS), kernel_size=(1, 1), stride=(1, 1))

In [23]:
params = {
    'learning_rate': 1.e-4,
    'batch_size': 1,
    'device': 'cpu'
}

In [24]:
mobilenet_train_and_val = ModelTrainAndVal(model, params, train_dataset, val_dataset, 'squeezenet')

In [25]:
mobilenet_train_and_val.train(1)

  0%|          | 0/506 [01:15<?, ?it/s]


KeyboardInterrupt: 